In [19]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 322 (delta 29), reused 15 (delta 15), pack-reused 278
Receiving objects: 100% (322/322), 11.71 MiB | 16.51 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [20]:
%cd shap-e

/content/shap-e/shap-e/shap-e


In [21]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/shap-e/shap-e/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-487yu42q/clip_d1578fed90224c7caeb30e7901ce8fba
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-487yu42q/clip_d1578fed90224c7caeb30e7901ce8fba
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e-0.0.0
  Running setup.py develop for shap-e


In [22]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [24]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [25]:
batch_size = 1
guidance_scale = 15.0
prompt = "apple"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [26]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAGYAAJkAAMwAADMzAGYzAJkzAMwzADNmAGZmAJlmAM…

In [27]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)